In [4]:
import os
import platform
import tkinter as tk
from tkinter import scrolledtext
import subprocess
import psutil
import socket



In [5]:
# ----------- OS Check -----------
def get_os_info():
    return platform.system()

# ----------- Get User Accounts -----------
def get_user_accounts():
    os_type = get_os_info()
    accounts = []

    if os_type == "Linux":
        with open("/etc/passwd") as f:
            for line in f:
                parts = line.strip().split(":")
                username = parts[0]
                uid = int(parts[2])
                shell = parts[-1]
                if uid >= 1000 and "nologin" not in shell:
                    accounts.append(username)
    elif os_type == "Windows":
        try:
            result = subprocess.check_output("net user", shell=True).decode(errors='ignore')
            lines = result.splitlines()
            start_collecting = False
            for line in lines:
                if "----" in line:
                    start_collecting = not start_collecting
                    continue
                if start_collecting:
                    if line.strip() == "" or "command completed" in line.lower():
                        break
                    accounts.extend(line.strip().split())
        except Exception as e:
            accounts.append(f"Error: {e}")

    return accounts




In [6]:
# ----------- Get Running Services Count -----------
def get_services_count():
    if get_os_info() == "Linux":
        result = subprocess.check_output("systemctl list-units --type=service --all", shell=True).decode()
        return len([line for line in result.splitlines() if ".service" in line])
    else:
        return len([s.name() for s in psutil.win_service_iter()])

# ----------- Log Reader -----------
def read_logs():
    logs = ""
    os_type = get_os_info()
    try:
        if os_type == "Linux":
            logs = subprocess.check_output("tail -n 100 /var/log/syslog", shell=True).decode()
        else:
            command = (
                'powershell "Get-EventLog -LogName System -Newest 50 | '
                'Select-Object TimeGenerated, EntryType, Source, Message | Format-Table -Wrap -AutoSize"'
            )
            logs = subprocess.check_output(command, shell=True).decode(errors="ignore")
    except Exception as e:
        logs = f"Log access failed: {e}"
    return logs



In [7]:
# ----------- Analyze Logs using Keyword Matching -----------
def analyze_logs(logs):
    suspicious_keywords = [
        'error', 'failed', 'denied', 'unauthorized', 'malicious',
        'attack', 'refused', 'critical', 'crash', 'unexpected', 'shutdown'
    ]

    if isinstance(logs, str):
        log_lines = logs.splitlines()
    else:
        log_lines = logs

    suspicious_logs = []
    for log in log_lines:
        lower_log = log.lower()
        if any(keyword in lower_log for keyword in suspicious_keywords):
            suspicious_logs.append(log)

    return "\n".join(suspicious_logs) if suspicious_logs else "✅ No suspicious entries found."



In [8]:
# ----------- Get Detailed Port Information -----------
def get_ports_info():
    connections = psutil.net_connections()
    port_details = {}

    suspicious_ports = {
        21: "FTP", 23: "Telnet", 135: "RPC", 139: "NetBIOS", 445: "SMB",
        1433: "MSSQL", 3306: "MySQL", 3389: "RDP", 5900: "VNC", 8080: "HTTP-Alt"
    }

    for conn in connections:
        laddr = conn.laddr
        if not laddr or not laddr.port:
            continue

        port = laddr.port
        proto = "TCP" if conn.type == socket.SOCK_STREAM else "UDP"
        status = conn.status
        pid = conn.pid
        try:
            proc = psutil.Process(pid) if pid else None
            proc_name = proc.name() if proc else "unknown"
        except:
            proc_name = "unknown"

        try:
            service_name = socket.getservbyport(port, proto.lower())
        except:
            service_name = "unknown"

        app_type = suspicious_ports.get(port, service_name.upper() if service_name != "unknown" else "Unknown")
        is_suspicious = "❌ Suspicious" if port in suspicious_ports else "✅ Safe"

        if port not in port_details:
            port_details[port] = {
                "protocol": proto,
                "service": service_name,
                "app_type": app_type,
                "statuses": set(),
                "proc_name": proc_name,
                "suspicious": is_suspicious
            }

        port_details[port]["statuses"].add(status)

    port_list = []
    for port, info in sorted(port_details.items()):
        statuses = ", ".join(info["statuses"])
        port_list.append(
            f"Port {port} ({info['service']}, {info['protocol']}, {info['app_type']}), "
            f"Process: {info['proc_name']} | {statuses} | {info['suspicious']}"
        )

    return port_list




In [9]:
# ----------- GUI Display -----------
def show_gui():
    window = tk.Tk()
    window.title("System Log and Port Analyzer")
    window.geometry("1000x800")

    os_info = get_os_info()
    accounts = get_user_accounts()
    service_count = get_services_count()
    logs = read_logs()
    analysis = analyze_logs(logs)
    port_list = get_ports_info()

    # System Info
    tk.Label(window, text=f"OS: {os_info}", font=("Helvetica", 11)).pack(pady=2)
    tk.Label(window, text=f"User Accounts ({len(accounts)}): {', '.join(accounts)}", font=("Helvetica", 10)).pack(pady=2)
    tk.Label(window, text=f"Running Services: {service_count}", font=("Helvetica", 10)).pack(pady=2)

    # Logs
    tk.Label(window, text="Raw System Logs", font=("Helvetica", 13, "bold")).pack(pady=10)
    log_box = scrolledtext.ScrolledText(window, height=10, width=120)
    log_box.insert(tk.END, logs)
    log_box.config(state='disabled')
    log_box.pack()

    # Analysis
    tk.Label(window, text="Suspicious Log Entries", font=("Helvetica", 13, "bold")).pack(pady=10)
    analysis_box = scrolledtext.ScrolledText(window, height=10, width=120)
    analysis_box.insert(tk.END, analysis)
    analysis_box.config(state='disabled')
    analysis_box.pack()

    # Port Info
    tk.Label(window, text="Open Ports and Status", font=("Helvetica", 13, "bold")).pack(pady=10)
    ports_box = scrolledtext.ScrolledText(window, height=10, width=120)
    ports_box.insert(tk.END, "\n".join(port_list) if port_list else "No open ports detected.")
    ports_box.config(state='disabled')
    ports_box.pack()

    tk.Button(window, text="Close", command=window.destroy).pack(pady=20)
    window.mainloop()

# ----------- Main -----------
if __name__ == "__main__":
    show_gui()